In [4]:
import torch
from torch import nn as nn
from torch import optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Heart_Disease_Prediction.csv")
df

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,Absence
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,Absence
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,Absence
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,Absence


In [6]:
df.drop(columns=["Sex","Chest pain type","FBS over 120",'EKG results','Exercise angina','Slope of ST','Number of vessels fluro'],inplace=True)
df

,Age,BP,Cholesterol,Max HR,ST depression,Thallium,Heart Disease
0,70,130,322,109,2.4,3,Presence
1,67,115,564,160,1.6,7,Absence
2,57,124,261,141,0.3,7,Presence
3,64,128,263,105,0.2,7,Absence
4,74,120,269,121,0.2,3,Absence
...,...,...,...,...,...,...,...
265,52,172,199,162,0.5,7,Absence
266,44,120,263,173,0.0,7,Absence
267,56,140,294,153,1.3,3,Absence
268,57,140,192,148,0.4,6,Absence


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Age            270 non-null    int64  
 1   BP             270 non-null    int64  
 2   Cholesterol    270 non-null    int64  
 3   Max HR         270 non-null    int64  
 4   ST depression  270 non-null    float64
 5   Thallium       270 non-null    int64  
 6   Heart Disease  270 non-null    object 
dtypes: float64(1), int64(5), object(1)
memory usage: 14.9+ KB


In [8]:
df.notna().sum()

,0
Age,270
BP,270
Cholesterol,270
Max HR,270
ST depression,270
Thallium,270
Heart Disease,270


In [9]:
df.isnull().sum()

,0
Age,0
BP,0
Cholesterol,0
Max HR,0
ST depression,0
Thallium,0
Heart Disease,0


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.describe()

,Age,BP,Cholesterol,Max HR,ST depression,Thallium
count,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000
mean,54.433333,131.344444,249.659259,149.677778,1.05000,4.696296
std,9.109067,17.861608,51.686237,23.165717,1.14521,1.940659
min,29.000000,94.000000,126.000000,71.000000,0.00000,3.000000
25%,48.000000,120.000000,213.000000,133.000000,0.00000,3.000000
50%,55.000000,130.000000,245.000000,153.500000,0.80000,3.000000
75%,61.000000,140.000000,280.000000,166.000000,1.60000,7.000000
max,77.000000,200.000000,564.000000,202.000000,6.20000,7.000000


In [12]:
df.shape

(270, 7)

In [13]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [14]:
X=X.values
X

array([[7.00e+01, 1.30e+02, 3.22e+02, 1.09e+02, 2.40e+00, 3.00e+00],
       [6.70e+01, 1.15e+02, 5.64e+02, 1.60e+02, 1.60e+00, 7.00e+00],
       [5.70e+01, 1.24e+02, 2.61e+02, 1.41e+02, 3.00e-01, 7.00e+00],
       ...,
       [5.60e+01, 1.40e+02, 2.94e+02, 1.53e+02, 1.30e+00, 3.00e+00],
       [5.70e+01, 1.40e+02, 1.92e+02, 1.48e+02, 4.00e-01, 6.00e+00],
       [6.70e+01, 1.60e+02, 2.86e+02, 1.08e+02, 1.50e+00, 3.00e+00]])

In [15]:
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [16]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y=y.reshape(-1,1)
y

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
    

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=40)
X_train


array([[ 48. , 130. , 256. , 150. ,   0. ,   7. ],
       [ 41. , 110. , 235. , 153. ,   0. ,   3. ],
       [ 52. , 118. , 186. , 190. ,   0. ,   6. ],
       ...,
       [ 64. , 180. , 325. , 154. ,   0. ,   3. ],
       [ 59. , 110. , 239. , 142. ,   1.2,   7. ],
       [ 44. , 120. , 220. , 170. ,   0. ,   3. ]])

In [18]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
X_train,X_test,y_train,y_test

(array([[-0.70746913, -0.07903635,  0.09731654,  0.00878829, -0.92852016,
          1.13643929],
        [-1.47647943, -1.16988955, -0.30544607,  0.14122572, -0.92852016,
         -0.92201678],
        [-0.26803467, -0.73354827, -1.2452255 ,  1.77462077, -0.92852016,
          0.62182527],
        ...,
        [ 1.0502687 ,  2.64809664,  1.4206794 ,  0.18537153, -0.92852016,
         -0.92201678],
        [ 0.50097563, -1.16988955, -0.22872938, -0.34437821,  0.11200049,
          1.13643929],
        [-1.14690359, -0.62446295, -0.59313365,  0.89170453, -0.92852016,
         -0.92201678]]),
 array([[-0.04831745, -1.16988955, -0.70820868,  0.36195478,  0.4588407 ,
         -0.92201678],
        [ 1.0502687 , -1.16988955, -0.7657462 , -0.25608659,  0.63226081,
         -0.92201678],
        [ 0.83055147, -0.62446295,  0.30828743, -2.24264813,  0.63226081,
          1.13643929],
        [-1.91591389, -0.07903635, -0.01775849,  1.64218333,  2.10633172,
         -0.92201678],
        [-0.597

In [20]:
X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
y_train_tensor = torch.tensor(y_train,dtype = torch.long)
y_test_tensor = torch.tensor(y_test,dtype = torch.long)

In [21]:
y_train_tensor

tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],

In [22]:
input_size = X_train_tensor.shape[1]
input_size

6

In [23]:
logistic_model = nn.Sequential(
    nn.Linear(input_size,1),             # Linear(y = x * w.T + b) ------>(Auto weight initialize W(2nd,1st)),here w.T(1st,2nd)
    nn.Sigmoid()
)

In [24]:
criterion = nn.BCELoss()
optimizer = optim.SGD(logistic_model.parameters(),lr = .001)

In [25]:
TrainDataset = TensorDataset(X_train_tensor.float(),y_train_tensor.float())
train_loader = DataLoader(TrainDataset,batch_size = 20,shuffle = True)

In [26]:

for epoch in range(1000):
    batch_loss = 0.00
    for X_batch,y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = logistic_model(X_batch)
        loss = criterion(y_pred,y_batch)        # this loss is the average loss per batch(Per individual row sample), so we need to multiply it by the batch size to get the total loss for that batch
        batch_loss = batch_loss + loss.item() * X_batch.size(0)  # item() converts tensor to float that stop extra computational graph from being created.
        loss.backward()
        optimizer.step()
    Total_loss_per_epoch = batch_loss/len(train_loader.dataset)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {Total_loss_per_epoch:.4f}")
        
        

Epoch 10, Loss: 0.8136
Epoch 20, Loss: 0.7832
Epoch 30, Loss: 0.7555
Epoch 40, Loss: 0.7303
Epoch 50, Loss: 0.7074
Epoch 60, Loss: 0.6866
Epoch 70, Loss: 0.6677
Epoch 80, Loss: 0.6505
Epoch 90, Loss: 0.6348
Epoch 100, Loss: 0.6205
Epoch 110, Loss: 0.6073
Epoch 120, Loss: 0.5954
Epoch 130, Loss: 0.5844
Epoch 140, Loss: 0.5744
Epoch 150, Loss: 0.5652
Epoch 160, Loss: 0.5567
Epoch 170, Loss: 0.5489
Epoch 180, Loss: 0.5416
Epoch 190, Loss: 0.5349
Epoch 200, Loss: 0.5287
Epoch 210, Loss: 0.5229
Epoch 220, Loss: 0.5175
Epoch 230, Loss: 0.5125
Epoch 240, Loss: 0.5079
Epoch 250, Loss: 0.5036
Epoch 260, Loss: 0.4995
Epoch 270, Loss: 0.4957
Epoch 280, Loss: 0.4922
Epoch 290, Loss: 0.4889
Epoch 300, Loss: 0.4857
Epoch 310, Loss: 0.4828
Epoch 320, Loss: 0.4800
Epoch 330, Loss: 0.4774
Epoch 340, Loss: 0.4750
Epoch 350, Loss: 0.4726
Epoch 360, Loss: 0.4705
Epoch 370, Loss: 0.4684
Epoch 380, Loss: 0.4664
Epoch 390, Loss: 0.4646
Epoch 400, Loss: 0.4628
Epoch 410, Loss: 0.4611
Epoch 420, Loss: 0.4596
E

In [27]:
# Make sure evaluation is done without gradients
with torch.no_grad():
    # Forward pass: get probabilities
    test_logits = logistic_model(X_test_tensor.float())  # shape: (num_samples, 1)
    
    # Convert probabilities to 0/1 predictions
    test_preds = (test_logits >= 0.5).float()            # still shape (num_samples, 1)
    
    # Flatten both prediction and true labels
    test_preds = test_preds.view(-1)                     # shape (num_samples,)
    y_test_flat = y_test_tensor.view(-1).float()        # shape (num_samples,)
    
    # Compute accuracy
    correct_test = (test_preds == y_test_flat).sum().item()
    test_accuracy = correct_test / y_test_flat.size(0)
    
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")


Test Accuracy: 70.37%


In [28]:
# Make sure evaluation is done without gradients
with torch.no_grad():
    # Forward pass: get probabilities
    train_logits = logistic_model(X_train_tensor.float())  # shape: (num_samples, 1)
    
    # Convert probabilities to 0/1 predictions
    train_preds = (train_logits >= 0.5).float()            # still shape (num_samples, 1)
    
    # Flatten both prediction and true labels
    train_preds = train_preds.view(-1)                     # shape (num_samples,)
    y_train_flat = y_train_tensor.view(-1).float()        # shape (num_samples,)
    
    # Compute accuracy
    correct_train = (train_preds == y_train_flat).sum().item()
    train_accuracy = correct_train / y_train_flat.size(0)
    
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")


Test Accuracy: 70.37%
